In [1]:
# SELF REMINDER. Copy the 'ocpa' directory to the forked one from github, so that I can push updates to github.

# Python native
import pickle
from statistics import median as median
from tqdm import tqdm
import os
os.chdir("c:\\Users\\Tim\\Development\\OCELFeatureExtractionExperiments")
from copy import deepcopy

# Data handling
import pandas as pd
import numpy as np

# Object centric process mining
from ocpa.algo.predictive_monitoring.obj import Feature_Storage as FeatureStorage

# PyG
import torch
from ocpa_PyG_integration.PyG_dataset import EventGraphDataset


c:\Users\Tim\Development\OCELFeatureExtractionExperiments\.env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 1.13.1+cpu
Cuda available: False
Torch geometric version: 2.2.0


In [2]:
def count_parameters(model) ->int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [3]:
ds_train = EventGraphDataset(
    root="data/ocpa-processed/",
    filename="BPI2017-feature_storage_split.pkl",
    label_key=("event_remaining_time", ()),
    train=True,
)

Processing...
22056it [01:07, 326.93it/s]
Done!


In [4]:
ds_test = EventGraphDataset(
    root="data/ocpa-processed/",
    filename="BPI2017-feature_storage_split.pkl",
    label_key=("event_remaining_time", ()),
    test=True,
)

Processing...
9453it [00:31, 302.11it/s]
Done!


In [7]:
ds_train.get_summary()

100%|██████████| 22056/22056 [00:54<00:00, 403.23it/s] 


EventGraphDataset (#graphs=22056):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |     12.5 |     13.7 |
| std        |      3.6 |      4.5 |
| min        |      6   |      5   |
| quantile25 |     10   |     11   |
| median     |     12   |     13   |
| quantile75 |     14   |     16   |
| max        |     41   |     54   |
+------------+----------+----------+

In [5]:
from model import GCN
import torch 
from torch_geometric.loader import DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from tqdm import tqdm


# Initialize model
model = GCN(ds_train.num_node_features, 12)
print(model)
print(f'Number of parameters: {count_parameters(model)}')


# Use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# data = ds_train.to(device)

# Initialize Optimizer
learning_rate = 0.01
decay = 5e-4
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=learning_rate, 
                             weight_decay=decay)
NUM_GRAPHS_PER_BATCH = 512
# Define loss function (CrossEntropyLoss for Classification Problems with 
# probability distributions)
criterion = torch.nn.MSELoss()

train_loader = DataLoader(ds_train,batch_size=NUM_GRAPHS_PER_BATCH,shuffle=True)
test_loader = DataLoader(ds_test,batch_size=NUM_GRAPHS_PER_BATCH,shuffle=True)

def calculate_metrics(y_pred, y_true, epoch, type):
    print(f"\n MAE: \n {mean_absolute_error(y_pred, y_true)}")
    print(f"MSE: {mean_squared_error(y_true, y_pred)}")
    print(f"R^2: {r2_score(y_true, y_pred)}")

I'm updated!
GCN(
  (conv1): GCNConv(23, 12)
  (conv2): GCNConv(12, 12)
  (out): Linear(in_features=12, out_features=1, bias=True)
)
Number of parameters: 457


In [6]:


def train(epoch):
      all_preds = []
      all_labels = []

      model.train()
      optimizer.zero_grad() 
      # Use all data as input, because all nodes have node features
      out = model(data.x, data.edge_index)  
      # Only use nodes with labels available for loss calculation --> mask
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  
      loss.backward() 
      optimizer.step()
      return loss

def train_one_epoch(epoch, model, train_loader, optimizer, loss_fn):
    # Enumerate over the data
    all_preds = []
    all_labels = []
    running_loss = 0.0
    step = 0
    for _, batch in enumerate(tqdm(train_loader)):
        # Use GPU
        batch.to(device)  
        # Reset gradients
        optimizer.zero_grad() 
        # Passing the node features and the connection info
        pred = model(batch.x.float(), 
                                batch.edge_index, 
                                batch.batch) 
        # Calculating the loss and gradients
        loss = loss_fn(torch.squeeze(pred), batch.y.float())
        loss.backward()  
        optimizer.step()  
        # Update tracking
        running_loss += loss.item()
        step += 1
        print(f'{running_loss} \r', end=" ")
        print(f'{step} \r', end=" ")
        all_preds.append(np.rint(torch.sigmoid(pred).cpu().detach().numpy()))
        all_labels.append(batch.y.cpu().detach().numpy())
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    
    calculate_metrics(all_preds, all_labels, epoch, "train")
    return running_loss/step

def test(epoch, model, test_loader, loss_fn):
    all_preds = []
    all_preds_raw = []
    all_labels = []
    running_loss = 0.0
    step = 0
    for batch in test_loader:
        batch.to(device)  
        pred = model(batch.x.float(), 
                        batch.edge_index, 
                        batch.batch) 
        loss = loss_fn(torch.squeeze(pred), batch.y.float())

         # Update tracking
        running_loss += loss.item()
        step += 1
        all_preds.append(np.rint(torch.sigmoid(pred).cpu().detach().numpy()))
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
        all_labels.append(batch.y.cpu().detach().numpy())
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    print(all_preds_raw[0][:10])
    print(all_preds[:10])
    print(all_labels[:10])
    calculate_metrics(all_preds, all_labels, epoch, "test")
    return running_loss/step

def run_training(loss_fn):
    
    # Start training
    best_loss = 1000
    early_stopping_counter = 0
    for epoch in range(300): 
        if early_stopping_counter <= 10: # = x * 5 
            # Training
            model.train()
            loss = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn)
            print(f"Epoch {epoch} | Train Loss {loss}")

            # Testing
            model.eval()
            if epoch % 5 == 0:
                loss = test(epoch, model, test_loader, loss_fn)
                print(f"Epoch {epoch} | Test Loss {loss}")
                
                # Update best loss
                if float(loss) < best_loss:
                    best_loss = loss
                    # # Save the currently best model 
                    # mlflow.pytorch.log_model(model, "model", signature=SIGNATURE)
                    early_stopping_counter = 0
                else:
                    early_stopping_counter += 1

        else:
            print("Early stopping due to no improvement.")
            return [best_loss]

In [7]:
results = run_training(criterion)

  0%|          | 0/44 [00:04<?, ?it/s]


TypeError: forward() takes 3 positional arguments but 4 were given